# read01 Crawler



## Outline





In [7]:
import os
import gzip
import re
import http.cookiejar
import urllib.request
import urllib.parse
from time import strftime
import time
import requests #HTTP for Humans 
from bs4 import BeautifulSoup
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
import datetime as dt

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

from urllib.parse import quote

import logging
from logging.handlers import RotatingFileHandler

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
loghandler = logging.handlers.TimedRotatingFileHandler(filename='logs/read01_crawler.log', when="midnight")
loghandler.setLevel(logging.INFO)
fileformatter = logging.Formatter('%(asctime)s - %(filename)s [%(levelname)s] >>> %(message)s')
loghandler.setFormatter(fileformatter)
logger.addHandler(loghandler)

In [4]:
header = {
'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4,zh-CN;q=0.2',
'Cache-Control':'max-age=0',
'Connection':'keep-alive',
'Cookie':'e2e68024-b067-4a05-88c2-d210988909cd; __atssc=google%3B2; revamp_ab=0; app_session=e2suaivqk8vucaq7b2b42il5o1; __cfduid=d7441027afe644fcf118db9936d8d39f71504669226; _ga=GA1.2.1377396399.1465715277; _gid=GA1.2.540087031.1504669228; __atuvc=5%7C32%2C3%7C33%2C0%7C34%2C1%7C35%2C2%7C36; __atuvs=59af6e2cc876cedd001',
'DNT':'1',
'Referer':'https://read01.com/search/?q=deep+learning',
'upgrade-insecure-requests':'1',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36', 
}
   

**   連結 **

In [5]:
def check_folderexist(path='img/'):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc: 
            logger.exception("message")
            if exc.errno != errno.EEXIST:
                raise    
                
def check_complete(folder):
    try:
        if 'complete.flag' in  os.listdir(folder):
            return True
        else:
            return False
    except:
        return False
    

def setcompleteflag(folder):
    path = '{folder}{file}'.format(folder=folder, file='complete.flag')
    file = open(path,'w') 
    file.write(str(dt.datetime.now()))
    file.close()
    
    
def crawler(folder, searchword= '深度學習'):
    driver =webdriver.PhantomJS(executable_path='/opt/phantomjs-2.1.1-macosx/bin/phantomjs')
    searchword_encode = quote(searchword.encode("utf-8"), safe='')
    url = 'https://read01.com/search/?q={w}'.format(w=searchword_encode)
    for i in range(1,11):
        print('topic {t}, page {i}'.format(t=searchword, i=i))
        retry_cnt=3
        while True:
            try:
                driver.get(url)
                driver.find_element_by_xpath("//div[@class='gsc-cursor']/div[{i}]".format(i=i)).click()
                time.sleep(3)
                soup = BeautifulSoup(driver.page_source, "html.parser")
                break
            except Exception as e:
                print(e)
                print('retry...')
                if retry_cnt>0:
                    time.sleep(20)
                    retry_cnt=retry_cnt-1
                    continue
                else:
                    break
        for link in soup.select('.gs-title.gsc-table-cell-thumbnail.gsc-thumbnail-left a'):
            try:
                if not link.has_attr('href'):
                    continue
                title = link.text
                href = link['href']
                print(i, title, href)
                r = sess.get(href, headers = header)
                soup_cont = BeautifulSoup(r.content, "html.parser")
                content_tag = soup_cont.find("div", attrs={'class':['content']})
                content_txt = content_tag.text
                filname='{y}'.format(y=title)
                filname = filname.strip()
                path ='{folder}{file}'.format(folder=folder, file=filname)
                file = open(path,'w') 
                file.write(content_txt) 
                file.close() 
            except Exception as e:
                print("error, {i},{f}".format(i=i, f=filname))



In [9]:
topics=['深度學習','Deep Learning','人工智慧','自動駕駛','hinton','百度','AI']
sess = requests.Session()
for topic in topics:
    folder = 'text/techdoc/{t}/'.format(t=topic)
    check_folderexist(folder)
    if not check_complete(folder):
        crawler(folder, topic)
        setcompleteflag(folder)



topic 人工智慧, page 1
1 這篇文章讓你看懂人工智慧：原理、技術和未來- 壹讀 https://read01.com/xynOoM.html
1 讓人驚懼的人工智慧- 壹讀 https://read01.com/PnD858.html
1 百度發布開源人工智慧代碼深度學習將更為簡單快速- 壹讀 https://read01.com/OynDBA.html
1 晶片GPU對未來人工智慧深度學習有多重要？ - 壹讀 https://read01.com/x75N.html
1 人工智慧簡史- 壹讀 https://read01.com/JoNNjK.html
1 未來人工智慧時代，需要什麼樣的人才- 壹讀 https://read01.com/Gg8Ma2.html
1 淺析認知心理學對人工智慧發展的影響- 壹讀 https://read01.com/K0PyE6.html
1 請問從弱人工智慧到強人工智慧，還有多遠？ - 壹讀 https://read01.com/DyB0Kn.html
1 人工智慧未來將呈現四大趨勢- 壹讀 https://read01.com/PBzAJN.html
1 人工智慧未來的發展方向- 壹讀 https://read01.com/EEGoDP.html
topic 人工智慧, page 2
2 人工智慧將滅絕人類？科學家告訴你真相- 壹讀 https://read01.com/M3PO.html
2 15款開源人工智慧軟體大盤點- 壹讀 https://read01.com/NPnJoJ.html
2 人類完敗, 人工智慧在醫學領域戰勝人類, 醫生將大規模下崗? - 壹讀 https://read01.com/JgGBRz.html
2 扎克伯格懟「鋼鐵俠」：預言人工智慧製造世界末日太不負責任- 壹讀 https://read01.com/zPPM767.html
2 動口不動手，暴風人工智慧電視X5ECHO教你玩- 壹讀 https://read01.com/ROzedM.html
2 人工智慧將怎樣改變保險業？ - 壹讀 https://read01.com/EQ43mk.html
2 李開復哥大畢業典禮演講：未來10年屬於人工智慧- 壹讀 https://read01.com/A3x54L.html
2 展望2017

3 阿波羅計劃：百度成為首家對外開放自動駕駛技術與的公司- 壹讀 https://read01.com/MPB86d.html
3 自動駕駛「不接鍋」 特斯拉被判無罪- 壹讀 https://read01.com/Q8Jae2.html
3 沃爾沃全新XC60細節曝光搭載最新自動駕駛系統- 壹讀 https://read01.com/AROEg8.html
3 無人駕駛汽車上路被撞翻Uber暫停自動駕駛項目- 壹讀 https://read01.com/DoRdRa.html
3 百度發布自動駕駛平台「阿波羅」，將開放感知、規劃和作業系統等代碼 ... https://read01.com/KmPeog.html
3 豐田最新自動駕駛測試車曝光，用了7枚雷射雷達- 壹讀 https://read01.com/nKNm67.html
3 現代Ioniq亮相CES 17：用戶買得起的自動駕駛汽車- 壹讀 https://read01.com/2PBkoL.html
3 特斯拉自動駕駛事故梳理：雷射雷達是否必要- 壹讀 https://read01.com/A5da8j.html
3 飛機都有自動駕駛是不是所有人都會開了呢- 壹讀 https://read01.com/RnEAoa.html
topic 自動駕駛, page 4
Message: {"errorMessage":"Unable to find element with xpath '//div[@class='gsc-cursor']/div[4]'","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Connection":"close","Content-Length":"117","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:60589","User-Agent":"Python http auth"},"httpVersion":"1.1","method":"POST","post":"{\"using\": \"xpath\", \"sessionId\": \"1f05c6f0-92f3-11e7-

9 自動駕駛將顛覆傳統汽車電子生態- 壹讀 https://read01.com/JdKG3P.html
9 深度學習&增強學習，自動駕駛新算法CP - 壹讀 https://read01.com/eMDe80.html
9 百度推「Apollo」計劃將開源自動駕駛技術- 壹讀 https://read01.com/RNjkk0.html
9 自動駕駛不等於無人駕駛認清自動駕駛的五個等級- 壹讀 https://read01.com/BNeEeg.html
9 密西根州通過自動駕駛法規允許自動駕駛汽車上路- 壹讀 https://read01.com/6AyEAg.html
9 自媒體札記：我們真的需要自動駕駛嗎？ - 壹讀 https://read01.com/55L08K.html
9 2020年成自動駕駛元年，NVIDIA/ST/NXP都有啥「寶貝」？ - 壹讀 https://read01.com/P67JnL.html
error, 9,2020年成自動駕駛元年，NVIDIA/ST/NXP都有啥「寶貝」？ - 壹讀
9 NVIDIA發布自動駕駛相關硬體和人工智慧技術- 壹讀 https://read01.com/52RDoL.html
9 自動駕駛汽車2020上市，發生車禍到底誰負責？ - 壹讀 https://read01.com/0aGK3m.html
topic 自動駕駛, page 10
10 加州車管局文檔公開蘋果自動駕駛測試車長這樣- 壹讀 https://read01.com/nBzdzE.html
10 自動駕駛在中國- 壹讀 https://read01.com/NPdDNJ.html
10 當我們談論自動駕駛時，我們談些什麼？ - 壹讀 https://read01.com/D58BPg.html
10 百度Apollo自動駕駛平台開放尋跡車亮相- 壹讀 https://read01.com/J0BOG5P.html
10 通用汽車首批130輛自動駕駛版雪佛蘭Bolt純電動車下線- 壹讀 https://read01.com/zA3dAj.html
10 蓋世汽車  這些自動駕駛汽車主流傳感器你真正了解的有多少? - 壹讀 https://read01.com/6M8gD8.html
10 自動駕駛將給車險行業帶來這三大衝擊- 壹讀 h

9 DBM 和DBN 的區別- 壹讀 https://read01.com/zh-tw/mE433j.html
9 學霸吳恩達的綜合素質，刷新「人生贏家」標準| 人物- 壹讀 https://read01.com/D43jog.html
topic hinton, page 10
10 矽谷AI名人錄：百度大腦吳恩達- 壹讀 https://read01.com/dkD3PN.html
10 【深度學習】深度學習在圖像識別中的研究進展與展望- 壹讀 https://read01.com/64AaM.html
10 淺析深度學習- 壹讀 https://read01.com/EGgJO4.html
10 全世界最兇殘的黑幫都在這裡- 壹讀 https://read01.com/MeLKQ3.html
10 Neural Networks for Machine Learning 課程筆記- 壹讀 https://read01.com/5o5B2.html
10 班夫國家公園－加斯伯國家公園6天遊記- 壹讀 https://read01.com/D8nMRd.html
10 深度學習系列：詳解工具箱參數及網絡一種改進方式-dropout - 壹讀 https://read01.com/E5zxLB.html
10 為什麼 Deep Learning 最先在語音識別和圖像處理領域取得突破 ... https://read01.com/MmyGj3.html
10 《實習醫生格蕾》第十四季將失去一名常規演員- 壹讀 https://read01.com/zh-hk/PO03oE.html
10 Deep Learning方向的paper整理- 壹讀 https://read01.com/NNexn.html
topic 百度, page 1
1 百度算法獨家揭秘- 壹讀 https://read01.com/K0kOg.html
1 那些年被百度賣過的遊戲貼吧- 壹讀 https://read01.com/QONeN6.html
1 海外充值百度雲會員方法，支持Paypal - 壹讀 https://read01.com/N2Joj3.html
1 百度四季度營利跌至歷史最低，陸奇：拿什麼拯救李彥宏- 壹讀 https://read01.com/2NmEMj.html
1 

8 百度正式宣布調整醫療事業部：​關停部分業務- 壹讀 https://read01.com/yGL8DM.html
8 百度2016年營收705億- 壹讀 https://read01.com/DmMNGP.html
topic 百度, page 9
9 百度推「Apollo」計劃將開源自動駕駛技術- 壹讀 https://read01.com/RNjkk0.html
9 百度網盤助手*改-破解百度下載限制- 壹讀 https://read01.com/7O03P7.html
9 谷歌安卓系統官方警告：百度手機助手不安全應卸載- 壹讀 https://read01.com/8z2N6P.html
9 百度全面啟動實名制！必須過這一關- 壹讀 https://read01.com/Kz6d6R.html
9 百度與谷歌比較：江湖郎中終難生存- 壹讀 https://read01.com/0AndJm.html
9 百度雲盤不用百度雲管家下載大文件的方法圖解- 壹讀 https://read01.com/gGBMAg.html
9 百度與中國農業銀行合作- 壹讀 https://read01.com/Qn7M2A.html
9 BT種子文件/電驢/迅雷/磁力連結分分鐘秒存百度網盤教程詳解- 壹讀 https://read01.com/dQBGkk.html
error, 9,BT種子文件/電驢/迅雷/磁力連結分分鐘秒存百度網盤教程詳解- 壹讀
9 智能語音一握即啟百度DuerOS強勢賦能HTC手機- 壹讀 https://read01.com/exQP2e.html
9 百度、騰訊、阿里三大巨頭誰在掉隊（值得一看） - 壹讀 https://read01.com/PogEo8.html
topic 百度, page 10
10 百度雲限速破解，兩種方法- 壹讀 https://read01.com/6dRknj.html
10 百度要全面實行實名制，你會進行身份驗證嗎？ - 壹讀 https://read01.com/MK0DNN.html
10 百度高層變動紛紜：吳恩達離職李彥宏「受困」 - 壹讀 https://read01.com/BmmA2M.html
10 百度網盤帳號要求實名認證官方給出詳細解讀- 壹讀 https://read01.com/Qmj0Re.

7 AI 會對宗教信仰產生威脅嗎？ - 壹讀 https://read01.com/jny7a0.html
7 象棋AI算法- 壹讀 https://read01.com/D5kOgg.html
topic AI, page 8
8 關注| 2017年百度聯盟大會：李彥宏闡述AI時代五大思維方式- 壹讀 https://read01.com/JnzQ88.html
8 一張圖看懂AI、機器學習和深度學習的區別- 壹讀 https://read01.com/B3JG7L.html
8 AI進階教程！手把手叫你繪製20個Apple Watch的圖標- 壹讀 https://read01.com/47K0Jz.html
8 如何使用AI製作萬花筒效果？ - 壹讀 https://read01.com/RzooOg.html
8 關於機器學習、深度學習和AI的完全初學者指南- 壹讀 https://read01.com/2eknyM.html
8 對AI毫無了解？本文帶你輕鬆了解AI - 壹讀 https://read01.com/g8PQ27.html
8 專訪《世界軍事》主編：AI取代戰機飛行員早著呢！ - 壹讀 https://read01.com/3QJjn4.html
8 2017要知道的7個AI應用趨勢- 壹讀 https://read01.com/mGL5e6.html
8 如何使用PS和AI製作蜂巢背景圖片？ - 壹讀 https://read01.com/APJjzM.html
8 騰訊任命張潼為AI Lab主任，劉熾平稱將加大AI投入- 壹讀 https://read01.com/j84KkD.html
topic AI, page 9
9 喜馬拉雅FM發布小雅AI音箱含海量內容- 壹讀 https://read01.com/n5BLkd.html
9 如何在Indesign導入能編輯的AI文件（兩種方法介紹） - 壹讀 https://read01.com/akGM7.html
9 盤點史上最全人工智慧產業：AI醫療每年能省540億美元- 壹讀 https://read01.com/2k86Q5.html
9 解決AI無法打開插圖窗口文件空間不足的方法- 壹讀 https://read01.com/m6z65o.html
9 全球AI報告Ⅳ：一年68億

1

***

*** END ***